In [49]:
# imports 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# sklearn imports
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

sns.set()
%matplotlib inline

In [3]:
df = pd.read_csv('data/cleaned.csv')
df.head()

,country,total_vaccinations_x,amount_of_days,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share,region,sub-region,vaccine,total_vaccinations_y,days_to_make_vaccines,amount_vaccines_needed
0,Afghanistan,422200.0,59,39074280,2.33 %,886592,60,652860,-62920.0,4.6,18,25 %,0.50 %,Asia,Southern Asia,0,0.0,0.0,0.0
1,Albania,11295390.0,59,2877239,-0.11 %,-3120,105,27400,-14000.0,1.6,36,63 %,0.04 %,Europe,Southern Europe,0,0.0,0.0,0.0
2,Algeria,75030.0,59,43984569,1.85 %,797990,18,2381740,-10000.0,3.1,29,73 %,0.56 %,Africa,Northern Africa,0,0.0,0.0,0.0
3,Andorra,185420.0,59,77287,0.16 %,123,164,470,0.0,N.A.,N.A.,88 %,0.00 %,Europe,Southern Europe,0,0.0,0.0,0.0
4,Angola,1551757.0,59,33032075,3.27 %,1040977,26,1246700,6413.0,5.6,17,67 %,0.42 %,Africa,Sub-Saharan Africa,0,0.0,0.0,0.0


In [10]:
columns = ['country', 'amount_of_days', 'Population (2020)', 'region', 'sub-region', 'total_vaccinations_y', 'total_vaccinations_x']

tmp_df = df[columns]
tmp_df.head()

,country,amount_of_days,Population (2020),region,sub-region,total_vaccinations_y,total_vaccinations_x
0,Afghanistan,59,39074280,Asia,Southern Asia,0.0,422200.0
1,Albania,59,2877239,Europe,Southern Europe,0.0,11295390.0
2,Algeria,59,43984569,Africa,Northern Africa,0.0,75030.0
3,Andorra,59,77287,Europe,Southern Europe,0.0,185420.0
4,Angola,59,33032075,Africa,Sub-Saharan Africa,0.0,1551757.0


In [12]:
le = LabelEncoder()

for col in tmp_df.columns.values:
    if tmp_df[col].dtype == 'object':
        tmp_df[col]=le.fit_transform(tmp_df[col])
        
tmp_df.head()

,country,amount_of_days,Population (2020),region,sub-region,total_vaccinations_y,total_vaccinations_x
0,0,59,39074280,2,12,0.0,422200.0
1,1,59,2877239,3,13,0.0,11295390.0
2,2,59,43984569,0,7,0.0,75030.0
3,3,59,77287,3,13,0.0,185420.0
4,4,59,33032075,0,14,0.0,1551757.0


In [52]:
X = tmp_df[['amount_of_days', 'Population (2020)', 'region', 'sub-region', 'total_vaccinations_y']]
y = tmp_df['total_vaccinations_x']
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)

In [53]:
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.20353943667834673

In [54]:
y_pred = model.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
138,4231.0,1.113310e+08
16,3634823.0,5.871312e+07
155,14142101.0,9.618301e+07
96,12639574.0,4.921488e+07
68,1923281.0,7.631448e+07
...,...,...
31,723834607.0,1.296129e+09
173,2500.0,1.019931e+08
12,47340398.0,6.134877e+07
109,3864618.0,7.399031e+07
